In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xllm-speech-ner-data/test_transcriptions.csv
/kaggle/input/xllm-speech-ner-data/dev_transcriptions.csv
/kaggle/input/xllm-speech-ner-data/transcriptions - transcriptions.csv
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/run.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/run.bash
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/README.md
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/requirements.txt
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/modelBuilder.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/sentence_splitter.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/utils.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/label_smoother_sum.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/constrained_seq2seq.py
/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/seq2seq/

In [2]:
!pip install transformers datasets accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 18.1 MB/s eta 0:00:00


In [3]:
train_df = pd.concat([pd.read_csv('/kaggle/input/xllm-speech-ner-data/transcriptions - transcriptions.csv'), pd.read_csv('/kaggle/input/xllm-speech-ner-data/dev_transcriptions.csv')], ignore_index=True)
train_df.sample(20)

,File Name,Transcription
5452,train-14629.wav,"John Kennedy, George H. Bush, and even John C..."
5434,train-11210.wav,Right?
17218,train-9686.wav,NaN
8484,train-9271.wav,They've been at this for almost three years.
13607,train-6071.wav,among the people that will be here to greet t...
16605,train-6407.wav,and the humor did remain the same.
271,train-17679.wav,I believe that any new statewide service facil...
7988,train-17194.wav,Scott
10045,train-10819.wav,"Um, the"
8754,train-16717.wav,What is strange is everyone knows who won. Thi...


In [4]:
import json

event_schema_json_path = '/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/challenge/event-schema.json'

with open(event_schema_json_path, 'r') as f:
    event_schema = json.load(f)

print(event_schema)

{'Event Type': ['Injure', 'Charge-Indict', 'Execute', 'Merge-Org', 'Start-Position', 'Die', 'Acquit', 'Extradite', 'Pardon', 'Convict', 'Phone-Write', 'End-Org', 'Release-Parole', 'Transfer-Money', 'Marry', 'Appeal', 'Fine', 'Be-Born', 'Transport', 'Sentence', 'Declare-Bankruptcy', 'Divorce', 'Meet', 'Demonstrate', 'Attack', 'Start-Org', 'Arrest-Jail', 'Elect', 'End-Position', 'Sue', 'Trial-Hearing', 'Nominate', 'Transfer-Ownership'], 'Argument Role': ['Instrument', 'Seller', 'Entity', 'Person', 'Attacker', 'Org', 'Beneficiary', 'Victim', 'Target', 'Buyer', 'Plaintiff', 'Agent', 'Artifact', 'Adjudicator', 'Defendant', 'Place', 'Prosecutor', 'Giver', 'Destination', 'Origin', 'Vehicle', 'Recipient'], 'Event Schema': {'Start-Org': ['Agent', 'Org', 'Place'], 'Die': ['Instrument', 'Place', 'Victim', 'Person', 'Agent'], 'Start-Position': ['Person', 'Entity', 'Place'], 'Acquit': ['Adjudicator', 'Defendant'], 'Trial-Hearing': ['Place', 'Prosecutor', 'Adjudicator', 'Defendant'], 'Attack': ['Ins

In [5]:
train_df['event-tag'] = ''

In [6]:
train_df.head(10)

,File Name,Transcription,event-tag
0,train-15746.wav,Joywin Seamer,
1,train-696.wav,We don't need war.,
2,train-18618.wav,"But, well, the business is complicated, the b...",
3,train-14425.wav,"It's crazy. Referring to Al Qaeda, Musharov s...",
4,train-17963.wav,"In short, Europeans are getting used to the i...",
5,train-18614.wav,"Back in the day, 2000 to 2003 or so, some of ...",
6,train-220.wav,We got him because the Pakistanis got him.,
7,train-7637.wav,So I went and volunteer for him and everything...,
8,train-7038.wav,"And then of course there is the 0G bone loss,...",
9,train-13957.wav,Those troops will work to restore and maintain...,


In [7]:
import json

train_event_json_path = '/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/challenge/data/ACE05EN/train.jsonl'

train_event = []
with open(train_event_json_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        if len(data['event']) != 0:
            train_event.append(data)

print(len(train_event))

3342


In [8]:
dev_event_json_path = '/kaggle/input/xllm-speech-ner-data/SpeechEE-main/SpeechEE-main/challenge/data/ACE05EN/dev.jsonl'
with open(dev_event_json_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        if len(data['event']) != 0:
            train_event.append(data)

print(len(train_event))

3669


In [9]:
event_dict = {f"{line['id']}.wav": line['event'] for line in train_event if line['event']}
train_df['event-tag'] = train_df['File Name'].map(event_dict)

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20118 entries, 0 to 20117
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   File Name      20118 non-null  object
 1   Transcription  19866 non-null  object
 2   event-tag      3669 non-null   object
dtypes: object(3)
memory usage: 471.6+ KB


In [11]:
train_df.sample(20)

,File Name,Transcription,event-tag
19119,train-9338.wav,Hillary is also part of...,NaN
4174,train-16206.wav,"PS clipboard now, Sean for the future.",NaN
914,train-7632.wav,"Yeah, that's right.",NaN
13584,train-15341.wav,If you want to make this,NaN
6194,train-8864.wav,B,NaN
14148,train-11426.wav,"Oh, I see.",NaN
8521,train-597.wav,I can't quite figure that out.,NaN
8151,train-11956.wav,"Yeah. We haven't... Yeah, I think... Eventual...",NaN
2801,train-15558.wav,She's in deep shit if you haven't already.,NaN
5839,train-12216.wav,NaN,NaN


In [12]:
train_df = train_df.dropna(subset=['event-tag'])

In [13]:
train_df = train_df.reset_index(drop=True)
train_df.head(20)

,File Name,Transcription,event-tag
0,train-696.wav,We don't need war.,"[{'trigger': 'war', 'type': 'Attack', 'argumen..."
1,train-7637.wav,So I went and volunteer for him and everything...,"[{'trigger': 'met', 'type': 'Meet', 'arguments..."
2,train-4145.wav,"that is the very joyous town of Palestine, We...","[{'trigger': 'come', 'type': 'Transport', 'arg..."
3,train-5564.wav,We will watch and listening very closely. The ...,"[{'trigger': 'visit', 'type': 'Transport', 'ar..."
4,train-13860.wav,Chinese foreign ministry spokesman Lu Jiancha...,"[{'trigger': 'talks', 'type': 'Meet', 'argumen..."
5,train-17297.wav,state law defines deadly force as that which i...,"[{'trigger': 'death', 'type': 'Die', 'argument..."
6,train-17863.wav,"Under Barguti, the time Zeen has also carried...","[{'trigger': 'bombings', 'type': 'Attack', 'ar..."
7,train-2722.wav,"And as we listened to John Kim's report, we c...","[{'trigger': 'landed', 'type': 'Transport', 'a..."
8,train-17053.wav,While the media used words like military styl...,"[{'trigger': 'slayings', 'type': 'Die', 'argum..."
9,train-7231.wav,Is it doing anything to tighten up security in...,"[{'trigger': 'attacked', 'type': 'Attack', 'ar..."


In [14]:
df = train_df

In [15]:
import pandas as pd
from datasets import Dataset
df['input_text'] = df['Transcription']
df['target_text'] = df['event-tag'].apply(lambda x: json.dumps(x))
df = df[['input_text', 'target_text']].dropna()
dataset = Dataset.from_pandas(df)

In [16]:
!pip install -q transformers datasets peft accelerate bitsandbytes trl
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [17]:
from huggingface_hub import login
login(token="") 

In [18]:
# access_token = ""
access_token = ""

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "BurgerTruck/text2event2"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

In [22]:
text = "The police arrested the suspect in New York on Monday."

In [23]:
inputs = tokenizer.encode("extract events: " + text, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=512)
extracted_event = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(extracted_event)

Arrest-Jail arrested Person suspect Place New York Agent police


In [24]:
extracted_event

'Arrest-Jail arrested Person suspect Place New York Agent police'

In [25]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_name = "BurgerTruck/text2event2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=False,        
    device_map="cpu"
)


model = prepare_model_for_kbit_training(model)


config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q", "v"],  
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 738,847,744 || trainable%: 0.1597


In [26]:
import ast
from tqdm import tqdm
import pandas as pd

def convert_df_to_event_data(df):
    data = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Converting"):
        text = row["Transcription"]
        event_raw = row["event-tag"]

        if isinstance(event_raw, str):
            try:
                event_list = ast.literal_eval(event_raw)
            except Exception as e:
                print(f"Skipping row due to error: {e}")
                continue
        else:
            event_list = event_raw

        event_texts = []
        trigger_args_words = []

        for event in event_list:
            event_str = f"Event: {event['type']}\nTrigger: {event['trigger']}"
            trigger_args_words.append(event['trigger'])

            arguments = event.get("arguments", [])
            for arg in arguments:
                if isinstance(arg, dict):
                    role = arg.get("role", "UnknownRole")
                    name = arg.get("name", "UnknownEntity")
                    trigger_args_words.append(name)
                    event_str += f"\n{role}: {name}"

            event_texts.append(event_str)

        combined_event_text = "\n\n".join(event_texts)

        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            text = " ".join(trigger_args_words)

        data.append({
            "text": text,
            "event": combined_event_text
        })

    return data



converted_data = convert_df_to_event_data(train_df)

Converting: 100%|██████████| 3669/3669 [00:00<00:00, 13458.24it/s]


In [27]:
train_df.loc[2]['event-tag']

[{'trigger': 'come',
  'type': 'Transport',
  'arguments': [{'name': 'jessica lynch', 'role': 'Artifact'},
   {'name': 'home', 'role': 'Destination'}]}]

In [28]:
converted_data[:3]

[{'text': "We don't need war.", 'event': 'Event: Attack\nTrigger: war'},
 {'text': 'So I went and volunteer for him and everything and I met Hillary and and Tipper and Owl and I almost got to meet him.',
  'event': 'Event: Meet\nTrigger: met\nEntity: I\nEntity: Hillary\nEntity: Tipper\nEntity: Al'},
 {'text': ' that is the very joyous town of Palestine, West Virginia on the news that Jessica Lynch is eventually going to come home.',
  'event': 'Event: Transport\nTrigger: come\nArtifact: jessica lynch\nDestination: home'}]

In [29]:
from datasets import Dataset


dataset = Dataset.from_list(converted_data)

def preprocess(example):
    input_text = "extract events: " + example["text"]
    target_text = example["event"]
    input_ids = tokenizer(input_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    target_ids = tokenizer(target_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    return {
        "input_ids": input_ids["input_ids"][0],
        "attention_mask": input_ids["attention_mask"][0],
        "labels": target_ids["input_ids"][0]
    }

dataset = dataset.map(preprocess)


Map:   0%|          | 0/3669 [00:00<?, ? examples/s]

In [30]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [31]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./text2event-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps = 20,
    save_strategy="epoch",               
    save_total_limit=3,                  
    learning_rate=2e-4,
    predict_with_generate=True,          
    report_to="none",                    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

<ipython-input-31-54c1fcb97dde>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [32]:
trainer.train(resume_from_checkpoint="checkpoint-260")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
20,9.375400
40,7.851300
60,5.261800
80,1.852200
100,0.731200
120,0.529300
140,0.456700
160,0.384600
180,0.328900
200,0.285000


KeyboardInterrupt: 

In [33]:
text = "The police arrested the suspect in New York on Monday."
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256)
event_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Predicted Event:")
print(type(event_text))

Predicted Event:
<class 'str'>


In [34]:
test_df = pd.read_csv('/kaggle/input/xllm-speech-ner-data/test_transcriptions.csv')
test_df.sample(10)

,File Name,Transcription
641,test-391.wav,"Police arrested her in Abelene, Texas, Saturd..."
302,test-668.wav,"Welch also wants decals on Jane Beasley, Welc..."
369,test-128.wav,A Pakistani court in Central Punjab province ...
493,test-367.wav,Putin insisted that the two sides were quote ...
579,test-59.wav,We are losing our positions in Asia because t...
54,test-348.wav,But pair flew to Singapore last year. After h...
645,test-67.wav,We were so proud of forming an anti-war block...
257,test-626.wav,"early Saturday, more units were waiting in Ku..."
663,test-438.wav,"With opposition hardening, the White House le..."
478,test-625.wav,Other units move into airfield complexes in W...


In [35]:
from tqdm import tqdm
import pandas as pd


test_df["Transcription"] = test_df["Transcription"].fillna("").str.lower()

# Function to generate prediction for a single sentence
def extract_event(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply generation over the entire DataFrame
predictions = []
for text in tqdm(test_df["Transcription"], desc="Generating events"):
    prediction = extract_event(text, model, tokenizer)
    predictions.append(prediction)

# Save predictions to new column
test_df["predicted_event"] = predictions


Generating events: 100%|██████████| 676/676 [2:12:51<00:00, 11.79s/it]  


In [36]:
test_df.sample(10)

,File Name,Transcription,predicted_event
286,test-224.wav,"on more, we'll be taken to the appeal court e...",Transport taken Artifact we Transport taken
261,test-501.wav,"since the same date for scientists, every fumes.",
601,test-79.wav,"putin wore face re-election in march 2004, an...",Elect re-election Elect re-election
210,test-535.wav,iraqi kurds fear that turkish troops could mo...,Transport move Transport move Transport move T...
221,test-653.wav,mr. campbell is sufficiently embarrassed and ...,
234,test-549.wav,if erdogan decides that for his own reasons a...,
454,test-594.wav,"the vendor also is holding talks with viacom,...",Transfer-Ownership acquiring Transfer-Ownershi...
163,test-39.wav,herriri has headed the country's government f...,End-Position headed Person herriri Start-
520,test-183.wav,the international novelty value with heidi pr...,
76,test-226.wav,but this is also conditional on police approval.,


In [37]:
np.sum(test_df["predicted_event"].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0) == True)

333

In [38]:
test_df.to_csv("results.csv", index=False)

In [39]:
model.save_pretrained("xllm-ner")
tokenizer.save_pretrained("tokenizer")
import shutil

# Zip model folder
shutil.make_archive("xllm-ner", 'zip', "xllm-ner")
shutil.make_archive("tokenizer", 'zip', "tokenizer")

'/kaggle/working/tokenizer.zip'

In [40]:
import time

while True:
    print("Keeping CPU busy...")
    time.sleep(300)

Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...
Keeping CPU busy...


KeyboardInterrupt: 